In [1]:
import numpy as np
import pandas as pd
import random
from keras.models import Model, load_model
from keras.layers import Input, Conv1D, BatchNormalization, add, Activation, MaxPooling1D, UpSampling1D
from keras.layers import AveragePooling1D, Concatenate
from keras.optimizers import Adam
import keras.backend as K
import tensorflow as tf
import matplotlib.pyplot as plt
from scipy.io import loadmat

Using TensorFlow backend.
/home/luke/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/luke/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/luke/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/luke/.local/lib/python3.6/site-packages/tensorflow/python/framewo

In [2]:
mat = loadmat('data/Filtered Data.mat')  # load mat-file
mdata = mat['Filtered']  # variable in mat file
mdtype = mdata.dtype  # dtypes of structures are "unsized objects"
# * SciPy reads in structures as structured NumPy arrays of dtype object
# * The size of the array is the size of the structure array, not the number
#   elements in any particular field. The shape defaults to 2-dimensional.
# * For convenience make a dictionary of the data using the names from dtypes
# * Since the structure has only one element, but is 2-D, index it at [0, 0]
ndata = {n: mdata[n][0, 0] for n in mdtype.names}

In [3]:
ndata

{'setname': array(['L04Level_075_resampled_filtered'], dtype='<U31'),
 'filename': array([], dtype='<U1'),
 'filepath': array([], dtype='<U1'),
 'subject': array([], dtype='<U1'),
 'group': array([], dtype='<U1'),
 'condition': array([], dtype='<U1'),
 'session': array([], shape=(0, 0), dtype=uint8),
 'comments': array(['Original file: C:\\Ll04\\EEG\\level_075.bdf'], dtype='<U40'),
 'nbchan': array([[128]], dtype=uint8),
 'trials': array([[1]], dtype=uint8),
 'pnts': array([[47100]], dtype=uint16),
 'srate': array([[150]], dtype=uint8),
 'xmin': array([[0]], dtype=uint8),
 'xmax': array([[313.99333333]]),
 'times': array([[0.00000000e+00, 6.66666667e+00, 1.33333333e+01, ...,
         3.13980000e+05, 3.13986667e+05, 3.13993333e+05]]),
 'data': array([[   545.6284 ,    544.1772 ,    543.83673, ...,    415.32474,
            413.08957,    412.30643],
        [  -200.56104,   -200.73323,   -200.93918, ...,   -380.60077,
           -381.54666,   -382.1281 ],
        [  -253.92542,   -254.28

In [4]:
cols = dict()
curr = 0
for chan in range(ndata['data'].shape[0]):
    string = "chan" + str(curr)
    cols[string] = ndata['data'][curr]
    curr += 1
df=pd.DataFrame(cols)

In [5]:
df

,chan0,chan1,chan2,chan3,chan4,chan5,chan6,chan7,chan8,chan9,...,chan118,chan119,chan120,chan121,chan122,chan123,chan124,chan125,chan126,chan127
0,545.628418,-200.561035,-253.925415,-18.295683,3142.251709,-1494.991821,-1302.116211,1877.775146,-19.010559,-118.778885,...,-2740.151123,-21492.525391,-24365.060547,-15475.704102,-4928.614258,1633.939941,-18142.179688,-11399.075195,-10475.907227,-330.469177
1,544.177185,-200.733231,-254.282593,-18.977837,3141.120850,-1495.413208,-1301.963501,1877.193970,-19.916296,-119.641785,...,-2740.908447,-21494.246094,-24369.695312,-15474.250977,-4930.812988,1635.763184,-18141.511719,-11399.955078,-10475.656250,-328.104767
2,543.836731,-200.939178,-254.515991,-18.961260,3139.559326,-1496.672241,-1302.526611,1876.401855,-20.794729,-120.817398,...,-2741.284180,-21488.507812,-24366.054688,-15470.814453,-4940.586914,1636.009521,-18135.929688,-11399.305664,-10472.332031,-325.871796
3,545.061035,-200.893951,-254.349335,-18.613771,3139.608643,-1497.180420,-1302.836304,1876.842041,-20.057074,-120.088509,...,-2733.850586,-21477.490234,-24358.968750,-15468.427734,-4952.375488,1636.405396,-18131.207031,-11396.824219,-10465.639648,-317.837738
4,546.501099,-200.349792,-254.441360,-18.909513,3141.414062,-1495.947998,-1301.663574,1879.230347,-17.955669,-117.453682,...,-2723.653320,-21466.857422,-24353.662109,-15466.381836,-4969.986328,1638.415771,-18128.410156,-11390.812500,-10458.389648,-305.974823
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47095,419.652191,-380.330383,-398.393402,-147.709198,3011.567627,-1615.860229,-1420.257568,2140.263184,-143.007126,-276.110840,...,-2619.091309,-22453.667969,-25077.378906,-15578.093750,-4957.060059,1355.427979,-18582.431641,-11557.288086,-10533.389648,-663.877136
47096,418.352173,-380.061096,-397.959625,-147.293945,3010.977051,-1616.328857,-1420.621826,2139.095215,-144.323410,-277.463806,...,-2625.231201,-22457.011719,-25085.253906,-15576.240234,-4943.583008,1352.428955,-18581.652344,-11559.130859,-10534.895508,-667.665649
47097,415.324738,-380.600769,-397.879700,-147.763916,3010.147705,-1616.174561,-1420.324707,2137.564941,-144.691666,-278.255890,...,-2628.370605,-22458.167969,-25090.658203,-15573.991211,-4944.609375,1352.636353,-18581.099609,-11559.549805,-10533.416016,-666.357483
47098,413.089569,-381.546661,-398.473267,-147.852127,3008.247559,-1616.630615,-1421.156860,2135.629150,-146.563263,-280.048157,...,-2629.542480,-22462.076172,-25088.917969,-15576.344727,-4963.132324,1353.075928,-18583.853516,-11560.249023,-10535.522461,-668.830017


In [8]:
avgs = df.apply(np.mean)
len(avgs.index)

128

In [9]:
for n in range(len(avgs.index)):
    df[avgs.index[n]] = df[avgs.index[n]] - avgs[n]

In [10]:
maxes = df.apply(np.max)
mins = df.apply(np.min)
for n in range(len(maxes)):
    maxes[n] = np.max([maxes[n], np.abs(mins[n])])
for n in range(len(maxes)):
    df[maxes.index[n]] = df[maxes.index[n]]/maxes[n]*256
df

,chan0,chan1,chan2,chan3,chan4,chan5,chan6,chan7,chan8,chan9,...,chan118,chan119,chan120,chan121,chan122,chan123,chan124,chan125,chan126,chan127
0,85.464859,153.125702,129.940567,135.316940,129.690445,137.272186,127.289940,-170.001175,123.287376,139.134583,...,2.298030,223.898895,195.156662,139.949219,-33.042061,207.426041,175.790253,151.041794,63.557976,154.242340
1,82.975624,152.761902,129.105576,133.650391,128.248825,136.577057,127.543625,-170.916016,122.210526,138.173248,...,2.132957,223.187744,192.654739,141.526840,-33.323151,209.651245,176.325073,150.076904,63.794151,156.008194
2,82.391655,152.326797,128.559937,133.690887,126.258209,134.500122,126.608162,-172.162903,121.166145,136.863541,...,2.051060,225.559326,194.620010,145.257751,-34.572670,209.951889,180.794464,150.789062,66.922333,157.675888
3,84.491653,152.422348,128.949539,134.539841,126.321075,133.661819,126.093689,-171.470001,122.043152,137.675568,...,3.671345,230.112793,198.445129,147.848938,-36.079746,210.435043,184.575775,153.510254,73.220055,163.676117
4,86.961731,153.571991,128.734406,133.817307,128.622620,135.694855,128.041870,-167.710526,124.541550,140.610947,...,5.894023,234.507248,201.309738,150.070099,-38.331154,212.888641,186.815170,160.102814,80.042503,172.535904
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47095,-130.616638,-226.674805,-207.792709,-180.851669,-36.904934,-62.115227,-68.970993,243.185699,-24.134171,-36.143986,...,28.685198,-173.332886,-189.365234,28.788191,-36.678631,-132.489075,-176.708206,-22.457079,9.465417,-94.762794
47096,-132.846512,-226.105881,-206.778641,-179.837173,-37.657795,-62.888287,-69.576111,241.347183,-25.699123,-37.651283,...,27.346897,-174.714828,-193.616287,30.800491,-34.955696,-136.149292,-176.084244,-24.477896,8.048361,-97.592239
47097,-138.039337,-227.246048,-206.591797,-180.985352,-38.715042,-62.633755,-69.082520,238.938354,-26.136948,-38.533722,...,26.662607,-175.192688,-196.533630,33.242184,-35.086910,-135.896179,-175.641693,-24.937319,9.440604,-96.615234
47098,-141.873230,-229.244446,-207.979416,-181.200851,-41.137337,-63.386074,-70.464928,235.891190,-28.362122,-40.530430,...,26.407177,-176.807922,-195.594208,30.687048,-37.454922,-135.359680,-177.846680,-25.704092,7.458381,-98.461845


In [11]:
df = df.astype(int)

In [12]:
static = df[0:751].copy()
walk = df[752:].copy()
walk

,chan0,chan1,chan2,chan3,chan4,chan5,chan6,chan7,chan8,chan9,...,chan118,chan119,chan120,chan121,chan122,chan123,chan124,chan125,chan126,chan127
752,72,182,166,171,136,133,125,-148,132,145,...,-6,210,180,101,-35,167,159,123,94,172
753,78,188,173,178,133,129,122,-152,129,142,...,-6,210,176,99,-35,165,157,122,94,171
754,75,189,172,176,130,126,119,-154,127,140,...,-8,207,172,95,-36,162,155,116,90,166
755,76,188,170,175,128,127,118,-156,125,138,...,-9,204,171,92,-37,161,153,113,85,163
756,82,188,171,176,128,126,118,-157,125,137,...,-10,203,172,92,-37,161,152,112,83,163
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47095,-130,-226,-207,-180,-36,-62,-68,243,-24,-36,...,28,-173,-189,28,-36,-132,-176,-22,9,-94
47096,-132,-226,-206,-179,-37,-62,-69,241,-25,-37,...,27,-174,-193,30,-34,-136,-176,-24,8,-97
47097,-138,-227,-206,-180,-38,-62,-69,238,-26,-38,...,26,-175,-196,33,-35,-135,-175,-24,9,-96
47098,-141,-229,-207,-181,-41,-63,-70,235,-28,-40,...,26,-176,-195,30,-37,-135,-177,-25,7,-98


In [36]:
pos = list(range(len(walk) // STEPS))
test = random.sample(pos, 20)
test

[27,
 114,
 60,
 62,
 97,
 109,
 104,
 64,
 50,
 89,
 74,
 78,
 67,
 75,
 46,
 71,
 42,
 101,
 85,
 37]

In [54]:
arrayList = []
for chan in walk.columns:
    arrayList.append(np.array(df[chan]))
d = np.array(arrayList)
a.shape[1] % STEPS

252

In [74]:
walk.iloc[9][1]

169

In [13]:
DEF_LR = .01
DEF_LR_DEC = .01
POOLS = [8,6,4,2]
STEPS = 384
NUM_FILTERS = 2
KERNEL_SIZE = 5
DILATION = 2

class ConvNetwork:
    
    def __init__(self, batches=20, alpha = DEF_LR, alpha_dec = DEF_LR_DEC, pools = POOLS, steps = STEPS,
                filts = NUM_FILTERS, kernel=KERNEL_SIZE, dilation = DILATION):
        self.batchSize = batches
        self.channels = 128
        self.steps = steps
        self.pools = pools
        self.alpha = alpha
        self.filters = filts
        self.kSize = kernel
        self.dRate = dilation
        self.alpha_decay = alpha_dec
        self.m = self.buildModel()
        
    def TrainModel(self, df):
        static = df[0:751].copy()
        walk = df[752:].copy()
        batches = len(walk) // self.steps
        slop = len(walk) % self.steps
        sbatches = (len(static) - self.steps) // (batches//2)
        strides = (batches // 2)
        wStart = np.random.randint(0,slop)
        sStart = np.random.randint(0,sbatches)
        print(strides, wStart, sStart, batches, sbatches)
        #create list of static arrays
        staticList = []
        for n in range(strides-1):
            curr = static[sStart:(sStart+self.steps)].copy()
            #print(curr.shape)
            currList = []
            for dat in range(len(curr)):
                currList.append(curr.iloc[dat])
            currArray = np.array(currList)
            staticList.append(currArray)
            sStart += sbatches
        #create list of walking arrays
        walkList = []
        for n in range(batches-1):
            curr = walk[wStart:(wStart+self.steps)].copy()
            currList = []
            for dat in range(len(curr)):
                currList.append(curr.iloc[dat])
            currArray = np.array(currList)
            walkList.append(currArray)
            wStart += self.steps
        #print(len(staticList), len(walkList))
        random.shuffle(staticList)
        random.shuffle(walkList)
        X_dat = []
        Y_dat = []
        nums = len(walkList)
        while nums > 0:
            choice = random.randint(0,100)
            if choice < 30:
                if len(staticList) > 0:
                    X_dat.append(staticList.pop())
                    Y_dat.append([[0,1]])
            else:
                X_dat.append(walkList.pop())
                Y_dat.append([[1,0]])
                nums -= 1
        X = np.array(X_dat)
        Y = np.array(Y_dat)
        print(Y.shape, X.shape)
        histories = self.m.fit(X, Y, batch_size=20, epochs=10, validation_split=.1)
        return histories
    
    def batchGenerator(self, df):
        while True:
            static = df[0:751].copy()
            walk = df[752:].copy()
            batchData = random.sample(dataList, self.batchSize)
            self.masks=list()
            batch_x = list()
            batch_y = list()
            self.num_erased = 0
            masks = list()
            for j in range(4):
                mask = np.ones((32,128))
                timeSteps = np.random.randint(10,20)
                self.num_erased += timeSteps
                timeSteps = np.random.choice(a=list(range(32)), size=timeSteps, replace=False)
                for time in timeSteps:
                    mask[time] = np.zeros(128)
                masks.append(mask)
            masks = np.array(masks)
            for fname in batchData:
                SA = ScoreAnalyzer(fname)
                roll = SA.transpose()
                measures = roll.shape[1]//8
                endMeasure=np.random.randint(4,(measures+1))
                yDat = roll[:,(8*endMeasure - 32):endMeasure*8,:]
                ipt = yDat.copy()
                ipt = ipt * masks
                batch_y.append(yDat)
                inst = np.concatenate((ipt, masks.copy()))
                self.masks.append(masks.copy())
                batch_x.append(inst)
            batch_x = np.array(batch_x)
            batch_y = np.array(batch_y)
            self.masks = np.array(self.masks)
            print(batch_x.shape, self.masks.shape, np.count_nonzero(batch_y), np.count_nonzero(self.masks))
            yield(batch_x,batch_y)
        
    def lossFunction(self, y_true, y_pred):
        masks = self.masks
        masks = 1. - masks
        num = np.count_nonzero(masks[:,:,:,0])
        masks = tf.convert_to_tensor(masks, dtype=tf.float32)
        logits = tf.math.log(y_pred)
        unred_err = tf.math.multiply(logits,y_true)
        masks = tf.math.multiply(masks,y_true)
        res = tf.math.multiply(masks, logits)
        print(self.num_erased)
        return -(tf.reduce_sum(res)/num)
        
    def buildModel(self):
        data = Input(shape=(self.steps, self.channels))
        featureMaps = []
        #Build Encoder
        pconv1 = Conv1D(self.filters, self.kSize, padding='same', dilation_rate=self.dRate)(data)
        pbn1 = BatchNormalization()(pconv1)
        pconv2 = Conv1D(self.filters, self.kSize, padding='same', dilation_rate=self.dRate)(pbn1)
        pbn2 = BatchNormalization()(pconv2)
        featureMaps.append(pbn2)
        lastIter = MaxPooling1D(self.pools[0])(pbn2)
        for i in range(1,len(self.pools)):
            conv1 = Conv1D(self.filters, self.kSize, padding='same', dilation_rate=self.dRate)(lastIter)
            bn1 = BatchNormalization()(conv1)
            conv2 = Conv1D(self.filters, self.kSize, padding='same', dilation_rate=self.dRate)(bn1)
            bn2 = BatchNormalization()(conv2)
            featureMaps.append(bn2)
            lastIter = MaxPooling1D(pool_size = self.pools[i])(bn2)
            
        #Intermediate Convolutions:
        post1 = Conv1D(self.filters, self.kSize, padding='same', dilation_rate=self.dRate)(lastIter)
        postbn1 = BatchNormalization()(post1)
        post2 = Conv1D(self.filters, self.kSize, padding='same', dilation_rate=self.dRate)(postbn1)
        lastIter = BatchNormalization()(post2)
        
        #Build Decoder:
        for i in range(len(self.pools)):
            ups = UpSampling1D(size=self.pools[3-i])(lastIter)
            c1 = Conv1D(self.filters, self.kSize, padding='same', dilation_rate=self.dRate)(ups)
            b1 = BatchNormalization()(c1)
            conc = Concatenate()([b1, featureMaps[3-i]])
            c2 = Conv1D(self.filters, self.kSize, padding='same', dilation_rate=self.dRate)(conc)
            b2 = BatchNormalization()(c2)
            c3 = Conv1D(self.filters, self.kSize, padding='same', dilation_rate=self.dRate)(b2)
            lastIter = BatchNormalization()(c3)
        
        #Classifier:
        segment = AveragePooling1D(pool_size=self.steps, strides=self.steps)(lastIter)
        output = Conv1D(self.filters, 1, activation='softmax', padding='same')(segment)
        model = Model(inputs=data, outputs=output)
        model.compile(optimizer="Adam", loss='binary_crossentropy', metrics=['acc'])
        return model

In [14]:
CN = ConvNetwork()
CN.m.summary()



Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 384, 128)     0                                            
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 384, 2)       1282        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 384, 2)       8           conv1d_1[0][0]                   
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, 384, 2)       22          batch_normalization_1[0][0]      
__________________________________________________________________________________________

In [15]:
hist = CN.TrainModel(df)

60 263 4 120 6
(167, 1, 2) (167, 384, 128)
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 150 samples, validate on 17 samples
Epoch 1/10
150/150 [==============================] - 6s 39ms/step - loss: 0.5480 - acc: 0.8400 - val_loss: 1.2558 - val_acc: 0.7647
Epoch 2/10
150/150 [==============================] - 0s 2ms/step - loss: 0.4579 - acc: 0.8933 - val_loss: 1.1072 - val_acc: 0.8235
Epoch 3/10
150/150 [==============================] - 0s 2ms/step - loss: 0.4012 - acc: 0.9200 - val_loss: 0.7666 - val_acc: 0.8235
Epoch 4/10
150/150 [==============================] - 0s 2ms/step - loss: 0.3548 - acc: 0.9400 - val_loss: 0.5795 - val_acc: 0.8235
Epoch 5/10
150/150 [==============================] - 0s 2ms/step - loss: 0.3292 - acc: 0.9733 - val_loss: 0.4006 - val_acc: 0.8235
Epoch 6/10
150/150 [==============================] - 0s 2ms/step - loss: 0.2942 - acc: 0.9800 - val_loss: 0.4128 - val_acc: 0.8235
Epoch 7/10
150/150 [====